# ROBOFISH

### Imports

In [ ]:
import FISH2_functions

### Find machine address
Use the below function to find the USB address and serial number of the connected devices, one device at a time.  
If you know the identifier of the FTDI chip of your device use it as input.  
Otherwise follow the instructions of the function and unplug and plug your device to find the address.  
  
When the search is successful add the serial number (preferred) or the USB COM port (alternative) to the data file.  
Open the data file with the `ROBOFISH_user_program.py` and add the info to the `ROBOFISH_System_datafile.yaml`.

In [ ]:
FISH2_functions.find_address()

# Initiate system

In [ ]:
import FISH2_functions

#Path to the  database that is used to keep track of the experiment data. 
#database is automatically generated by the "FISH2_user_program.py" and prints the path after runing, copy the path to this location 
db_path = 'FISH_database\FISH_System2_db.sqlite'

#System specific path to start_imaging_file. If not present make a file called start_imaging_file.txt with a single 0.
#Then paste the path here, and also use this path for the imaging program.
start_imaging_file_path = "C:\\Users\\BL\\Desktop\\ROBOFISH\\start_imaging_file.txt"

#System specific path to where the microscope saves the images. The info files will be put here.
#Use double slashes "\\" but do not put the trailing slashes! Example "C:\\Folder\\subfolder"
imaging_output_folder = "G:\\To_Monod\\LBEXP20210428_EEL_HE_3370um"


F2 = FISH2_functions.FISH2(db_path, imaging_output_folder, start_imaging_file_path, system_name='ROBOFISH')

# Basic functions

The below commands are for basic operation of the ROBOFISH system.  
Some functions are capable of more complex settings. Please see the documentation of each function for a description.  
This information is accessible in Jupyter Lab by hitting Shift+Tab while having the cursor between the brackets of the function.  
Or in the source file `FISH2_functions.py`  
  
Volumes are in microliter (ul)  
Temperatures are in degree Celcius (C)  

In [ ]:
# Write message to log file
F2.L.logger.info('Your message here')

In [ ]:
# Reset the reservoir. Will refresh the requested amount of running buffer
F2.resetReservoir(100, update_buffer=True) 
    #Refreshing 100ul and updating the volume.

In [ ]:
# Dispense some running buffer to the flow cell
F2.extractDispenseRunningBuffer(1000, 'Chamber1') 
    #Will dispence 1000ul to Chamber1

In [ ]:
# Dispese a specfic buffer to the flow cell
F2.extractDispenseBuffer('buffer_name', 1000, 'Chamber1', padding=True)
    # "buffer_name" can be the name of the buffer or the port number.
    # Volume is 1000ul
    # Dispence to "Chamber1"
    # If padding is True the bufffer will be dispenced exactly to the flow cell.
    # The dead volume in the tubes between valve and flow cell will be bridged with running buffer.

In [ ]:
# Dispense a hybridization mix with probes to the flow cell
F2.extractDispenseHybmix('Chamber1', 3, slow_speed=32, steps=0)
    # Dispence hybridization mix beloing to Chamber1 cycle 3 to Chamber1.
    # Because the hybmix is viscous the pumping speed should be slower, here speed setting 32 is used.
    # Check the respective code of the pump for the correct speed code.
    # The steps option can have the hybridization mix be pumped intermittendly through the degasser.
    # Use this if you get issues with air bubles. Alternatively you can lower the speed even more.

In [ ]:
# Set the temperature of the flow cell
F2.setTemp(37, 'Chamber1')
    # Set temperature of Chamber1 to 37C.
    # If you want to acces the temperatures that you entered in the info file use the Parameters dictionary:
    # F2.Parameters['Imaging_temperature']

In [ ]:
# Wait untill a specific temperature has been reached.
F2.waitTemp(37, 'Chamber1', error=3, sd=0.01, verbose=False)
    # Blocks untill 37C has been reached in Chamber1.
    # The measured temperature may deviate 3C from the target temperature.
    # The standard deviation (sd) should be below 0.01 for the temperature to be considered stable.
    # If verbose is True is will print the measured temperature each second.

In [ ]:
# The extractDispence* functions can also take arguments that control the incubation time.
# They take the keyword agruments "h" for hours, "m" for minutes and "s" for seconds.
# Example:
F2.extractDispenseBuffer('buffer_name', 1000, 'Chamber1', padding=True, h=1, m=30)
    # This means that after the dispensing of the buffer the program will wait 1 hour and 30 minutes.
    # In other words the sample is incubated 1 hour 30 minutes with the dispensed buffer.
# The time it takes to execute the function is substracted from the incubation time if it does not take more than 10% of the incubation time.

In [ ]:
# To send an image to the microscope to start the imaging use the following function
F2.startImaging('Chamber1', start_imaging_file_path)
    # This waits untill the "start_imaging_file" is set to 0 and then sets it to 1 for 'Chamber1' 

In [ ]:
# This function explicitly waits untill the "start_imaging_file" is set to zero
F2.waitIimaging(start_imaging_file_path)

In [ ]:
# If your protocol needs to wait for something or you want to safely put your progam in idle use this function:
F2.secure_sleep(3600, period=120, alarm_room_temperature=35, temperature_range=5, number_of_messages=10)
    # This function blocks for 3600 seconds.
    # However every 120 seconds it wakes up and checks if there are any errors on the machines.
    # It also checks if the room temperature is not above 35C which could mean that the the room is on fire (or it is just a good day to go to the beach).
    # Furthermore, it checks if the connected Chambers are withing 5C from their set temperature.
    # And it also checks if the dist of the computer is not getting too full.
    # If it detects an error it will send the user 10 messages. This will wake you up hopefully if something is wrong.
    # It also checks if errors resolve themselve, in which case the user will be notified.
    # If it does not resolve the user will get 10 messages every 120 seconds untill it is fixed. 

In [ ]:
# Use the above funtions to build a protocol.
# For osmFISH and EEL the protocol consists of a first round and then by a number of repeated rounds.
# You can build this as two functions.
#Example:
def first_round(chamber, cycle_number):
    #Your protocol for first round here
    print('Completed first round')
    
def repeat_round(chamber, cycle_number):
    #Your protocol for the repeat round here
    print(f'Completed cycle {cycle_number}')

In [ ]:
# If you have your two functions that take a chamber and cycle number argument,
# you can use the scheduler function to execute the full experiment for you.
F2.scheduler(first_round, repeat_round)
    # This function will first perform the first_round() function
    # Then it will send a message to the microscope to start the imaging. (This needs to be set up, see ROBOFISH github page)
    # When the imaging is done it will perform the repeat_round() function.
    # The total number of rounds and other settings are controlled by the info_file.

# Advanced functions

### Accessing parameters

In [ ]:
# To access parameters that were given in the info file use the following dictionaries:
F2.Parameters #Most usefull info file parameters will be found here.
F2.Volumes #Current volumes of the buffers
F2.Targets #Gene/barcode names for all rounds.
F2.Ports #Name of the buffers connected to the ports.
F2.Ports_reverse #Name of the pots connected to the buffers.
F2.Hybmix #Hybmix codes connected to the ports.
F2.Machines #Which machines are active
F2.Machine_identification #Machine ID for identification
F2.Fixed_USB_port #USB port numbers for machines
F2.Operator_address #Pushbullet addresses of the operators
F2.Padding #Padding volumes
F2.Alert_volume #Allert volumes of buffers, waste and disk space.

In [ ]:
# Input for the functions can be hard coded like this:
F2.setTemp(37, 'Chamber1')

#But if your protocol has many "setTemp()" commands and you want to change them all it is easier to take the value from the Parameters dictionary:
F2.setTemp(F2.Parameters['Staining_temperature'], 'Chamber1')
    # If in the info file 'Staining_temperature' is set to 37 it will set the temperature of Chamber1 to 37C
    
#Another usefull parameter is the volume of the flow cell that can be used to determine the volume of the hybridization mix and washes.
F2.Parameters['Hybmix_volume']

In [ ]:
# Before the execution of many of the above functions the most up to date info is fetched from the info file.
# However, if you want to explicitly update the above dictionaries use this function:
F2.updateExperimentalParameters(F2.db_path)

### Communication with user

In [1]:
# To send a message to the user use the "push()" function.
subject = 'Update'
message = 'Experiment is running. Just sit back and relax. Maybe call that one friend that you wanted to call for weeks now. Yours faithfully -ROBOFISH'
F2.push(short_message = subject, long_message = message)

In [ ]:
# It is also possible to receive messages and do something with the reply. 
#(Carefull this can be error-prone if not implemented correctly, please read the doc string)
reply = F2.get_push(F2.Operator_address, operator='Operator1')

### Cleaning

In [ ]:
# This function helps the user to clean the ROBOFISH system.
F2.cleanSystem()

In [ ]:
# Function to clean the tubbing of one of the hybridization mixes.
# This function is also part of the "cleanSystem(") function and is incorporated in the "extractDispenseHybmix()" function.
F2.cleanHybmixTube('HYB01', 3)
    # Clean the tubbing of 'HYB01' 3 times.

### Low level functions
The above basic fluid handling functions are made with low level commands to the devices.  
(There are even lower level functions to control the pump, please refer to the code of the pump or the below functions if you require more control.)

In [ ]:
# Get the port number of a target buffer.
F2.getPort('WB')
    # Returns the port number connected to the 'WB' buffer.

In [ ]:
# Connect the valve to a specific port OR buffer.
F2.connectPort('P3')
    # Connect to port 3 on Valve1.
    
F2.connectPort('P14')
    # Connect Valve1 one to the port that connects to Valve2.
    # Connect Valve2 to port 4
    
F2.connectPort('WB')
    # Connect the valves so that the reservoir is connected to the 'WB' buffer.
    
F2.connectPort('Waste')
    # Connect the valves so that the reservoir is connected to the waste.  

In [ ]:
# Exctract a certain buffer into the reservoir.
F2.extractBuffer('WB', 950)
    # Exctract 950ul of 'WB' into the reservoir

In [ ]:
# Add padding volume to reservoir.
padding_volume = F2.padding('Chamber1')
    # Adds the padding volume related to the target, in this case 'Chamber1' to the reservoir.
    # This function will return the volume in microliter.

In [ ]:
# Dispense a certain volume to a target.
volume = 950 + padding_volume
F2.dispenseBuffer('Chamber1', volume)
    # Dispenses the full volume to the target 'Chamber1'
    # In this case the aspirated volume of 950ul and the padding volume from the above examples are dispenced .

In [ ]:
# To zero the pump you can use the following function.
F2.resetReservoir(replace_volume=0)
    # If the pump had aspirated anything, the pump will be emptied by dispensing it to the Waste.
    
# If you want to prevent contamination between buffers or just want to wash the reservoir you can increase the "replace_volume".
F2.resetReservoir(replace_volume=200)
    # This first empties the pump, similar to the "replace_volume=0" setting.
    # Then it aspirates 200ul of running buffer, and subsequently dispenses this to the Waste.

In [ ]:
# To explicitly check for errors use:
F2.check_error(alarm_room_temperature=35, temperature_range=5, number_of_messages=10)
    # Checks if there are any errors on the active machines. (Active means that they have a 1 in the Machines table of the info file.)
    # Checks if the room temperature is not above the "alarm_room_temperature".
    # Checks if the connected Chambers are withing 5C of the set temperature.
    # Checks if there is enough disk space for the images on the computer.
    # If it finds an error it will send the user 10 messages.
    # If there was an error in a previous check which is now resolved it will notifiy the user of this.

# This function is also incorporated in the "secureSleep()" function.

### Function wrapper

In [ ]:
# Important functions of the ROBOFISH system are wrapped with a wrapper so that functions are only executed when they can and use the latest experimental data:
# The wrapper executes the following functions in this order:
# - Checks if the experiment is Paused by the user. This can either be an explicit pause or when the user is updating the info file.
# - Updates the experimental parameters by fetching them form the info file.
# - Prime the buffers if the user instructed this.
# - Execute the wrapped function.
# - Secure sleep the incubation time if the user has given this. Meaning that the system will check for errors during the time it blocks.


# Functions are wrapped like this:

@functionWrap
def new_function(buffer, volume, target):
    F2.extractDispenseBuffer(buffer, volume, target)
    

# The wrapper takes incubation time arguments as "h" for hours, "m" for minutes and "s" for seconds
new_function('WB', 1000, 'Chamber1', h=1, m=30)
    # This function now does all the above items:
    # Check for paused experiment.
    # Update parameters.
    # Prime buffers
    # Then it executes the function and thus dispences 1000ul of 'WB' to 'Chamber1'
    # Then it secure sleeps for 1 hour and 30 minutes. Basically incubating the sample in Chamber1 for 1.5 hours with 'WB'. However it will check for errors.
    
# The following basic functions are wrapped like this:
F2.extractDispenseRunningBuffer()
F2.extractDispenseBuffer()
F2.extractDispenseHybmix()